# Zero-shot feedback from LLaMa3-8B-Instruct

In [1]:
# load required libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import psutil
import os
import random

In [ ]:
# Check available memory
memory = psutil.virtual_memory()
available_memory_gb = memory.available / (1024 ** 3)  # Convert to GB
print(f"Available Memory: {available_memory_gb} GB")

In [3]:
with open('.../huggingface_api_key.txt', 'r') as file: # load your api key
    api_key = file.read().replace('\n', '')

In [ ]:
# set model id
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# load pretrained tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          token=api_key,
                                          force_download=False)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
    tokenizer.model_max_length = 4096

# load pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             token=api_key,
                                             force_download=False)



### Define helperfunctions to sample essays from the folder and receive feedback for them from LLaMa3

In [115]:
def get_llama3_feedback(sample_student_essay: bool = True,
                        prompt: str = "Phones and driving",
                        student_essay: str = os.listdir("student_essays/")[0],
                        ):
    """
    Function to receive a feedback from llama3 on a given student essay.
    Input: Defines whether to sample a random essay from the folder (sample_student_essay=True) or provide feedback for a specific essay from the folder (to bedefined in the student_essay argument when sample_student_essay=False)
    Output: List containing student essay ID and trespective textfeedback from llama3.
    """
    
    valid_prompts = ['Phones and driving', 'Seeking multiple opinions', 'Summer projects']
    prompt_id = {'Phones and driving': 'In1_', 'Summer projects': 'In3_', 'Seeking multiple opinions': 'In15_'}
    system_prompts = {"Phones and driving": "You are a teacher who evaluatesstudent essays on the following argumentative writing task: 'Today the majority of humans own and operate cell phones on a daily basis. In essay form, explain if drivers should or should not be able to use cell phones in any capacity while operating a vehicle.' Provide the student with helpful feedback on his or her essay.",
                      "Summer projects": "You are a teacher who evaluatesstudent essays on the following argumentative writing task: 'Some schools require students to complete summer projects to assure they continue learning during their break. Should these summer projects be teacher-designed or student-designed? Take a position on this question. Support your response with reasons and specific examples.'. Provide the student with helpful feedback on his or her essay.",
                      "Seeking multiple opinions": "You are a teacher who evaluatesstudent essays on the following argumentative writing task: 'When people ask for advice, they sometimes talk to more than one person. Explain why seeking multiple opinions can help someone make a better choice. Use specific details and examples in your response.'. Provide the student with helpful feedback on his or her essay."}

    # check for valid prompt
    if prompt not in valid_prompts:
        raise InvalidPromptError(f"Invalid prompt: '{prompt}'. Valid prompts are: {', '.join(valid_prompts)}")    

    # sample essay from folder
    essay_id, student_essay = get_essay(sample_from_folder=sample_student_essay, prompt=prompt,file=student_essay)

    # create input messages
    messages = [ {"role": "system", "content": system_prompts[prompt]},
                 {"role": "user", "content": student_essay},
                ]
    # tokenize input
    input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
    ).to(model.device)
    
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    # get answer from llama3
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    
    response = outputs[0][input_ids.shape[-1]:]

    return [essay_id, tokenizer.decode(response, skip_special_tokens=True)]

In [109]:
# Define InvalidPromptError
class InvalidPromptError(Exception):
    """Custom exception for invalid prompts."""
    pass


def get_essay(sample_from_folder: bool = True, dir_to_essays: str = "student_essays/", prompt: str = "Phones and driving", file: str = essays[0], ):
    """
    Function which samples an essay from the given folder. 
    Available prompts are 'Phones and driving', 'Seeking multiple opinions' and 'Summer projects'.
    If sample_from_folder is True, the function returns the essay given in the 'file' argument.
    """
    essay_list = os.listdir(dir_to_essays)
    valid_prompts = ['Phones and driving', 'Seeking multiple opinions', 'Summer projects']
    prompt_id = {'Phones and driving': 'In1_', 'Summer projects': 'In3_', 'Seeking multiple opinions': 'In15_'}
    
    if prompt not in valid_prompts:
        raise InvalidPromptError(f"Invalid prompt: '{prompt}'. Valid prompts are: {', '.join(valid_prompts)}")
    
    if sample_from_folder:
        essay = random.choice([e for e in essay_list if prompt_id[prompt] in e])
        with open(dir_to_essays + essay) as f:
            essay1 = f.read()
    else:
        with open(dir_to_essays + file) as f:
            essay1 = f.read()    
    return [essay if sample_from_folder else file, essay1]


### Apply the functions

In [113]:
# test functionality
essay_id, student_essay = get_essay(sample_from_folder=False, prompt='Seeking multiple opinions')
print(f"Student essay ID: {essay_id}\n")
print(f"Student essay:\n {student_essay}\n")

Student essay ID: In3_AD3821286500.txt

*Student essay:*
 Summer, as known throughout the whole world, is for relaxation. Now when it comes to students, summer is a school free zone, mentally and physically. Though some teachers believe it is a smart idea to give the students projects or summer reading so they wouldn't forget some of the school work and already have a review over the summer when they come back. That is always expected when schools start back, though not all the students that were assigned to do the assignments did them because they either didn't understand the task correctly or they just wanted to enjoy summer and not think about any school work. That causes both the students and the teachers to be irritated and dissapointed. However, that could be changed by just one small upgrade.

For years, school systems and teachers have been assigning students lessons that they will either never use in the real life or lessons the students don't understand or don't want to learn

In [116]:
essay_id, feedback= get_llama3_feedback()
print(f"Student essay ID: {essay_id}\n")
print(f"Feedback for the student:\n {feedback}\n")

Student essay ID: In1_0B2E6B68B571.txt

Feedback for the student:
 Overall, your essay presents a clear and concise argument against drivers using cellphones while operating a vehicle. Here are some specific strengths and weaknesses to consider:

Strengths:

* You provide a clear thesis statement that sets the tone for the rest of the essay.
* You use specific statistics from the National Safety Council to support your argument, which adds credibility to your claims.
* You effectively explain how cellphone use can be a distraction and a habit for many people, which is a compelling point.
* You acknowledge the importance of cellphones in modern life, which shows that you have considered multiple perspectives on the issue.

Weaknesses:

* Your essay could benefit from more development and elaboration on your points. For example, you mention that distraction is a huge part of car crashes, but you don't provide any additional information or examples to support this claim.
* You could also 